# 应用单节点数据缓存

作者：MindSpore团队、[陈超然](https://gitee.com/sunny_ccr)

`Linux`  `Ascend` `GPU` `CPU` `数据准备` `中级` `高级`

[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/r1.7/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/r1.7/docs/notebook/mindspore_enable_cache.ipynb)

## 概述

对于需要重复访问远程的数据集或需要重复从磁盘中读取数据集的情况，可以使用单节点缓存算子将数据集缓存于本地内存中，以加速数据集的读取。

下面，本教程将演示如何使用单节点缓存服务来缓存经过数据增强处理的数据。

## 配置环境

使用缓存服务前，需要安装MindSpore，并设置相关环境变量。以Conda环境为例，需要完成`LD_LIBRARY_PATH`与`PATH`环境变量的配置

In [1]:
import os
import sys
import mindspore

python_path = "/".join(sys.executable.split("/")[:-1])
mindspore_path = "/".join(mindspore.__file__.split("/")[:-1])
mindspore_lib_path = os.path.join(mindspore_path, "lib")

if 'PATH' not in os.environ:
    os.environ['PATH'] = python_path
elif python_path not in os.environ['PATH']:
    os.environ['PATH'] += ":" + python_path
print(os.environ['PATH'])

os.environ['LD_LIBRARY_PATH'] = "{}:{}".format(mindspore_path, mindspore_lib_path)
print(os.environ['LD_LIBRARY_PATH'])

[WARNING] ME(4968:139868289333056,MainProcess):2021-02-25-22:00:42.129.964 [mindspore/ops/operations/array_ops.py:2302] WARN_DEPRECATED: The usage of Pack is deprecated. Please use Stack.


/usr/local/cuda/bin:/home/sunny/miniconda3/envs/seb/bin:/home/sunny/miniconda3/condabin:/usr/local/cuda-10.1/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin
/home/sunny/miniconda3/envs/seb/lib/python3.7/site-packages/mindspore:/home/sunny/miniconda3/envs/seb/lib/python3.7/site-packages/mindspore/lib


## 启动缓存服务器

在使用单节点缓存服务之前，首先需要启动缓存服务器：

In [2]:
!cache_admin --start

Cache server startup completed successfully!
The cache server daemon has been created as process id 5005 and listening on port 50052

Recommendation:
Since the server is detached into its own daemon process, monitor the server logs (under /tmp/mindspore/cache/log) for any issues that may happen after startup



若提示找不到`libpython3.7m.so.1.0`文件，尝试在虚拟环境下查找其路径并设置环境变量：

```bash
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:{path_to_conda}/envs/{your_env_name}/lib
```

## 创建缓存会话

若缓存服务器中不存在缓存会话，则需要创建一个缓存会话，得到缓存会话id：

In [3]:
!cache_admin -g

Session created for server on port 50052: 4173327275


缓存会话id由服务器随机分配。

## 创建缓存实例

在脚本中使用`DatasetCache` API来定义一个名为`some_cache`的缓存实例，并把上一步中创建的缓存会话id传入`session_id`参数：

In [4]:
import mindspore.dataset as ds

some_cache = ds.DatasetCache(session_id=4173327275, size=0, spilling=False)

## 插入缓存实例

下面样例中使用到CIFAR-10数据集。运行样例前，需要参照数据集加载中的方法下载并存放CIFAR-10数据集。目录结构如下：

```text
├─my_training_script.py
└─cifar-10-batches-bin
    ├── batches.meta.txt
    ├── data_batch_1.bin
    ├── data_batch_2.bin
    ├── data_batch_3.bin
    ├── data_batch_4.bin
    ├── data_batch_5.bin
    ├── readme.html
    └── test_batch.bin
```

在应用数据增强算子时将所创建的`some_cache`作为其`cache`参数传入：

In [5]:
import mindspore.dataset.vision.c_transforms as c_vision

dataset_dir = "cifar-10-batches-bin/"
data = ds.Cifar10Dataset(dataset_dir=dataset_dir, num_samples=5, shuffle=False, num_parallel_workers=1)

# apply cache to map
rescale_op = c_vision.Rescale(1.0 / 255.0, -1.0)
data = data.map(input_columns=["image"], operations=rescale_op, cache=some_cache)

num_iter = 0
for item in data.create_dict_iterator(num_epochs=1):  # each data is a dictionary
    # in this example, each dictionary has a key "image"
    print("{} image shape: {}".format(num_iter, item["image"].shape))
    num_iter += 1

0 image shape: (32, 32, 3)
1 image shape: (32, 32, 3)
2 image shape: (32, 32, 3)
3 image shape: (32, 32, 3)
4 image shape: (32, 32, 3)


通过cache_admin --list_sessions命令可以查看当前会话有五条数据，说明数据缓存成功。

In [6]:
!cache_admin --list_sessions

Listing sessions for server on port 50052

     Session    Cache Id  Mem cached Disk cached  Avg cache size  Numa hit
  4173327275   575278224           5         n/a           12442         5


## 销毁缓存会话

在训练结束后，可以选择将当前的缓存销毁并释放内存：

In [7]:
!cache_admin --destroy_session 4173327275

Drop session successfully for server on port 50052


以上命令将销毁缓存会话id为4173327275的缓存。

## 关闭缓存服务器

使用完毕后，可以选择关闭缓存服务器，该操作将销毁当前服务器中存在的所有缓存会话并释放内存。

In [8]:
!cache_admin --stop

Cache server on port 50052 has been stopped successfully.
